In [ ]:
import codenamesLLM
import pandas as pd
import openpyxl
from tqdm import tqdm

c:\Users\miche\OneDrive\Desktop\Codenames Progetto PDE\Codenames-LLM\codenames_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# PLAY AGGREGATE GAMES

In [ ]:
# Path to your Excel file
file_path = r"experiment_data\n_agents_data\agents_data.xlsx" #compy the model_tournament_input to use one new

# Read the Excel file into a DataFrame
df = pd.read_excel(file_path)

# Iterate through the rows using titled columns
for index, row in tqdm(df.iterrows(), total=len(df)):
    # Access values using column titles
    red_team = row['red_agents']  # Replace with the actual column title for the red team
    blue_team = row['blue_agents']  # Replace with the actual column title for the blue team
    red_cot = row['red_cot']
    blue_cot = row['blue_cot']
    playable = True
    already_played = pd.notna(row['winner'])

    if ((pd.notna(red_team) and pd.notna(blue_team)) and not(already_played)) and playable:  # Check if both values are not NaN
        try:
            # Call your function and get the result
            result = codenamesLLM.play_game(red_agents=red_team, red_cot_guesser=red_cot, blue_agents=blue_team, blue_cot_guesser=blue_cot)
            print("winner:",f"{red_team}({red_cot} cot)" if result[0] == "RED" else f"{blue_team}({blue_cot} cot) ","-> ", result[1])
            red_stats = codenamesLLM.analyze_team_guesses(result[3], "RED")
            blue_stats = codenamesLLM.analyze_team_guesses(result[3], "BLUE")

            df.at[index, 'red_agents'] = red_team
            df.at[index, 'red_cot'] = red_cot
            df.at[index, 'blue_agents'] = blue_team
            df.at[index, 'blue_cot'] = blue_cot
            df.at[index, 'winner'] = result[0]
            df.at[index, 'red_avg_words_2guess'] = red_stats['average_expected_guesses']
            df.at[index, 'blue_avg_words_2guess'] = blue_stats['average_expected_guesses']
            df.at[index, 'red_avg_words_guessed'] = red_stats['average_correct_guesses']
            df.at[index, 'blue_avg_words_guessed'] = blue_stats['average_correct_guesses']
            df.at[index, 'reason'] = result[1]
            df.at[index, 'red_turns'] = red_stats['total_hints']
            df.at[index, 'blue_turns'] = blue_stats['total_hints']
            df.at[index, 'red_cib'] = result[4]
            df.at[index, 'blue_cib'] = result[5]

        except Exception as e:
            print(f"skipped game: {e}")

    # Write the updated DataFrame back to the same Excel file
    df.to_excel(file_path, index=False)


 25%|██▌       | 30/120 [01:12<08:44,  5.82s/it]

winner: 5(False cot) ->  killer word selected


 35%|███▌      | 42/120 [01:24<02:35,  1.99s/it]

winner: 3(False cot)  ->  killer word selected


 42%|████▏     | 50/120 [04:44<17:55, 15.36s/it]

...

 83%|████████▎ | 100/120 [13:53<16:13, 48.68s/it]

winner: 5(False cot)  ->  cards finished


 88%|████████▊ | 106/120 [17:33<09:00, 38.63s/it]

winner: 2(False cot) ->  killer word selected


100%|██████████| 120/120 [17:33<00:00,  8.78s/it]


# TOURNAMENT DATA PROCESS

In [ ]:
import pandas as pd

def process_tournament_data(input_path, output_path):
    # Load the data
    df = pd.read_excel(input_path)

    # Function to calculate metrics
    def calculate_metrics(group):
        wins = group['winner'] == group['role'].str.upper()
        losses = ~wins
        card_finished = group['reason'] == 'cards finished'
        killer_word = group['reason'] == 'killer word selected'

        return {
            "model_name": group["model"].iloc[0],
            "games_played": len(group),
            "wins": wins.sum(),
            "win_percentage": 100 * wins.sum() / len(group),
            "win_by_cards_finished": (wins & card_finished).sum(),
            "wins_by_killer_words": (wins & killer_word).sum(),
            "losses_by_card_finished": (losses & card_finished).sum(),
            "losses_by_killer_words": (losses & killer_word).sum(),
            "average_word_to_guess": group['avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins": group.loc[wins, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_lose": group.loc[losses, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins_by_ending_cards": group.loc[wins & card_finished, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_loses_by_ending_cards": group.loc[losses & card_finished, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins_by_killer_card": group.loc[wins & killer_word, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_loses_by_killer_card": group.loc[losses & killer_word, 'avg_words_2guess'].mean(),
            "average_word_guessed": group['avg_words_guessed'].mean(),
            "average_word_guessed_when_wins": group.loc[wins, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_lose": group.loc[losses, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_wins_by_ending_cards": group.loc[wins & card_finished, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_loses_by_ending_cards": group.loc[losses & card_finished, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_wins_by_killer_card": group.loc[wins & killer_word, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_loses_by_killer_card": group.loc[losses & killer_word, 'avg_words_guessed'].mean(),
            "average_turns": group['turns'].mean(),
            "average_turns_when_wins": group.loc[wins, 'turns'].mean(),
            "average_turns_when_lose": group.loc[losses, 'turns'].mean(),
            "average_turns_when_wins_by_ending_cards": group.loc[wins & card_finished, 'turns'].mean(),
            "average_turns_when_loses_by_ending_cards": group.loc[losses & card_finished, 'turns'].mean(),
            "average_turns_when_wins_by_killer_cards": group.loc[wins & killer_word, 'turns'].mean(),
            "average_turns_when_loses_by_killer_cards": group.loc[losses & killer_word, 'turns'].mean(),
            "total_cib": group['cib'].sum()
        }

    # Reshape the dataset to treat roles equivalently
    red_df = df.rename(columns=lambda x: x.replace('red_', '')).assign(role='red', model=df['red_model'])
    blue_df = df.rename(columns=lambda x: x.replace('blue_', '')).assign(role='blue', model=df['blue_model'])
    combined_df = pd.concat([red_df, blue_df], ignore_index=True)

    # Metrics for models as a whole
    overall_metrics = combined_df.groupby("model").apply(calculate_metrics).apply(pd.Series)

    # Metrics for models playing as Red
    red_metrics = red_df.groupby("model").apply(calculate_metrics).apply(pd.Series)
    red_metrics['role'] = 'red'

    # Metrics for models playing as Blue
    blue_metrics = blue_df.groupby("model").apply(calculate_metrics).apply(pd.Series)
    blue_metrics['role'] = 'blue'

    # Save the results to an Excel file with three sheets
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        overall_metrics.to_excel(writer, sheet_name='Overall', index=False)
        red_metrics.to_excel(writer, sheet_name='Red', index=False)
        blue_metrics.to_excel(writer, sheet_name='Blue', index=False)


In [ ]:
input_path = 'experiment_data\model_tournament\model_tournament.xlsx'  # Replace with your input file path
output_path = 'experiment_data\model_tournament\model_tournament_stats.xlsx'  # Replace with your output file path

# Run the process
process_tournament_data(input_path, output_path)

# COT DATA PROCESS

In [ ]:
import pandas as pd

def process_tournament_data_by_cot(input_path, output_path):
    # Load the data
    df = pd.read_excel(input_path)

    # Function to calculate metrics
    def calculate_metrics(group):
        wins = group['winner'] == group['role'].str.upper()
        losses = ~wins
        card_finished = group['reason'] == 'cards finished'
        killer_word = group['reason'] == 'killer word selected'

        return {
            "cot": group["cot"].iloc[0],
            "games_played": len(group),
            "wins": wins.sum(),
            "win_percentage": 100 * wins.sum() / len(group),
            "win_by_cards_finished": (wins & card_finished).sum(),
            "wins_by_killer_words": (wins & killer_word).sum(),
            "losses_by_card_finished": (losses & card_finished).sum(),
            "losses_by_killer_words": (losses & killer_word).sum(),
            "average_word_to_guess": group['avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins": group.loc[wins, 'avg_words_2guess'].mean(),
            "average_word_to_guess_when_lose": group.loc[losses, 'avg_words_2guess'].mean(),
            "average_word_guessed": group['avg_words_guessed'].mean(),
            "average_word_guessed_when_wins": group.loc[wins, 'avg_words_guessed'].mean(),
            "average_word_guessed_when_lose": group.loc[losses, 'avg_words_guessed'].mean(),
            "average_turns": group['turns'].mean(),
            "average_turns_when_wins": group.loc[wins, 'turns'].mean(),
            "average_turns_when_lose": group.loc[losses, 'turns'].mean(),
            "total_cib": group['cib'].sum()
        }

    # Reshape the dataset to unify red and blue roles
    red_df = df.rename(columns=lambda x: x.replace('red_', '')).assign(role='red', cot=df['red_cot'])
    blue_df = df.rename(columns=lambda x: x.replace('blue_', '')).assign(role='blue', cot=df['blue_cot'])
    combined_df = pd.concat([red_df, blue_df], ignore_index=True)

    # Metrics for cot
    overall_metrics = combined_df.groupby("cot").apply(calculate_metrics).apply(pd.Series)

    # Metrics for cot in Red role
    red_metrics = red_df.groupby("cot").apply(calculate_metrics).apply(pd.Series)
    red_metrics['role'] = 'red'

    # Metrics for cot in Blue role
    blue_metrics = blue_df.groupby("cot").apply(calculate_metrics).apply(pd.Series)
    blue_metrics['role'] = 'blue'

    # Save the results to an Excel file with three sheets
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        overall_metrics.to_excel(writer, sheet_name='Overall', index=False)
        red_metrics.to_excel(writer, sheet_name='Red', index=False)
        blue_metrics.to_excel(writer, sheet_name='Blue', index=False)


In [ ]:
# Example usage:
input_path = 'experiment_data\cot_data\cot_data.xlsx'
output_path = 'experiment_data\cot_data\cot_data_stats.xlsx'
process_tournament_data_by_cot(input_path, output_path)

C:\Users\miche\AppData\Local\Temp\ipykernel_28900\2111975892.py:41: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  overall_metrics = combined_df.groupby("cot").apply(calculate_metrics).apply(pd.Series)
C:\Users\miche\AppData\Local\Temp\ipykernel_28900\2111975892.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  red_metrics = red_df.groupby("cot").apply(calculate_metrics).apply(pd.Series)
C:\Users\miche\Ap

# AGENTS NUMBER DATA PROCESS


In [ ]:
import pandas as pd

def process_agents_data(input_path, output_path):
    # Load the data
    df = pd.read_excel(input_path)

    # Function to calculate metrics
    def calculate_metrics(group, role):
        prefix = 'red_' if role == 'red' else 'blue_'
        wins = (group['winner'] == role.upper())
        losses = ~wins
        card_finished = group['reason'] == 'cards finished'
        killer_word = group['reason'] == 'killer word selected'

        return {
            "number_of_agents": group[f"number_{role}_agents"].iloc[0],
            "games_played": len(group),
            "wins": wins.sum(),
            "win_percentage": 100 * wins.sum() / len(group),
            "win_by_cards_finished": (wins & card_finished).sum(),
            "wins_by_killer_words": (wins & killer_word).sum(),
            "losses_by_card_finished": (losses & card_finished).sum(),
            "losses_by_killer_words": (losses & killer_word).sum(),
            "average_word_to_guess": group[f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins": group.loc[wins, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_lose": group.loc[losses, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins_by_ending_cards": group.loc[wins & card_finished, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_loses_by_ending_cards": group.loc[losses & card_finished, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_wins_by_killer_card": group.loc[wins & killer_word, f'{prefix}avg_words_2guess'].mean(),
            "average_word_to_guess_when_loses_by_killer_card": group.loc[losses & killer_word, f'{prefix}avg_words_2guess'].mean(),
            "average_word_guessed": group[f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_wins": group.loc[wins, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_lose": group.loc[losses, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_wins_by_ending_cards": group.loc[wins & card_finished, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_loses_by_ending_cards": group.loc[losses & card_finished, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_wins_by_killer_card": group.loc[wins & killer_word, f'{prefix}avg_words_guessed'].mean(),
            "average_word_guessed_when_loses_by_killer_card": group.loc[losses & killer_word, f'{prefix}avg_words_guessed'].mean(),
            "average_turns": group[f'{prefix}turns'].mean(),
            "average_turns_when_wins": group.loc[wins, f'{prefix}turns'].mean(),
            "average_turns_when_lose": group.loc[losses, f'{prefix}turns'].mean(),
            "average_turns_when_wins_by_ending_cards": group.loc[wins & card_finished, f'{prefix}turns'].mean(),
            "average_turns_when_loses_by_ending_cards": group.loc[losses & card_finished, f'{prefix}turns'].mean(),
            "average_turns_when_wins_by_killer_cards": group.loc[wins & killer_word, f'{prefix}turns'].mean(),
            "average_turns_when_loses_by_killer_cards": group.loc[losses & killer_word, f'{prefix}turns'].mean(),
            "total_cib": group[f'{prefix}cib'].sum()
        }

    # Metrics for agents playing as Red
    red_metrics = df.groupby("number_red_agents").apply(lambda g: calculate_metrics(g, "red")).apply(pd.Series)
    red_metrics['role'] = 'red'

    # Metrics for agents playing as Blue
    blue_metrics = df.groupby("number_blue_agents").apply(lambda g: calculate_metrics(g, "blue")).apply(pd.Series)
    blue_metrics['role'] = 'blue'

    # Combine overall metrics
    overall_metrics = pd.concat([red_metrics, blue_metrics]).reset_index(drop=True)

    # Save the results to an Excel file with three sheets
    with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
        overall_metrics.to_excel(writer, sheet_name='Overall', index=False)
        red_metrics.to_excel(writer, sheet_name='Red', index=False)
        blue_metrics.to_excel(writer, sheet_name='Blue', index=False)



In [ ]:
# Define input and output file paths
input_path = 'path_to_your_input_file.xlsx'  # Replace with your input file path
output_path = 'path_to_your_output_file.xlsx'  # Replace with your output file path

# Run the process
process_agents_data(input_path, output_path)